# Flickr8k Image Captioning Training (Colab Pro)

This notebook facilitates training the `mini-transformer` model on Google Colab.

### Professional Workflow:
1. **Mount Drive:** Saves trained model (`.pth`) and dataset zips persistently so we don't lose progress or have to re-download 1GB every time.
2. **Clone Repo:** Pulls latest code from GitHub into the Colab runtime.
3. **Hybrid Data Load:** Automatically detects if the dataset is on Google Drive; if not, it downloads it once and saves a copy.
4. **Install & Run:** Sets up the environment and starts the training script.

In [ ]:
import os
from google.colab import drive
from pathlib import Path

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Define Paths
GDRIVE_DATA_DIR = Path('/content/drive/MyDrive/trainingData')
LOCAL_DATA_DIR = Path('/content/ImageDescript/data')

# 3. Clone Repository
if not os.path.exists('ImageDescript'):
    !git clone https://github.com/aclink88/ImageDescript
%cd ImageDescript

# 4. Dynamic Data Loading
GDRIVE_DATA_DIR.mkdir(parents=True, exist_ok=True)
LOCAL_DATA_DIR.mkdir(parents=True, exist_ok=True)

def sync_data():
    zips = ['Flickr8k_Dataset.zip', 'Flickr8k_Text.zip']
    
    for zip_name in zips:
        gdrive_path = GDRIVE_DATA_DIR / zip_name
        local_zip_path = Path('/content') / zip_name
        
        if gdrive_path.exists():
            print(f"Found {zip_name} on Google Drive. Copying to local SSD...")
            !cp "{gdrive_path}" "{local_zip_path}"
        else:
            print(f"{zip_name} not found on Drive. Downloading from source...")
            url = f"https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/{zip_name}"
            !wget -O "{local_zip_path}" "{url}"
            print(f"Saving {zip_name} to Google Drive for future use...")
            !cp "{local_zip_path}" "{gdrive_path}"
        
        print(f"Extracting {zip_name} to local runtime disk...")
        !unzip -q "{local_zip_path}" -d "{LOCAL_DATA_DIR}"
        !rm "{local_zip_path}"
    
    # Handle the known typo in the original image zip folder name
    typo_dir = LOCAL_DATA_DIR / 'Flicker8k_Dataset'
    correct_dir = LOCAL_DATA_DIR / 'Flickr8k_Dataset'
    if typo_dir.exists() and not correct_dir.exists():
        typo_dir.rename(correct_dir)

sync_data()

# 5. Install Dependencies
!pip install spacy tqdm pandas Pillow torch torchvision
!python -m spacy download en_core_web_sm

# 6. Start Training
!python -m src.train.train

# 7. Save trained model back to Google Drive
if os.path.exists('captioning_model.pth'):
    # Ensure the destination directory exists
    !mkdir -p /content/drive/MyDrive/savedModels
    # Copy the model
    !cp captioning_model.pth /content/drive/MyDrive/savedModels/captioning_model.pth
    print("Model checkpoint successfully backed up to Google Drive at /savedModels/")